Populate Silver.Workflow table with data

In [0]:
%sql

drop table silver.workflow;
create table  silver.workflow as 
--Create and join CTEs to get the different values for the isMailing field (0,1) on a single record in order to perform a coalesce statement based on them since that is not possible if the values are on different records. 
				--Context: occasionally there are records that have a different State ID based on the isMailing value, and these then tie to different regions. In order to get the correct region, we have to do a conditional join to the State table based on the isMailing value, prioritizing State value where isMailing is 0. The Regions table is then joined twice, once to get the region listed on the File, which may be different and is used in the coalesce as a back up value, and then again to get the region associated with the State.
with isMailing0 as
(select ismailing as isMailing0, agencyid as agencyid0, State as State0 from bronze.tb_am_AgencyAddresses_spark where ismailing =0),

isMailing1 as
(select ismailing as isMailing1, agencyid as agencyid1, State as State1 from bronze.tb_am_AgencyAddresses_spark where ismailing=1),

joinIsMailing as
(select * from ismailing0 full outer join ismailing1 on ismailing0.agencyid0=ismailing1.agencyid1)


SELECT
--Fact Data
trim(fpol.PolicyNumber) as PolicyNumber
,trim(whtasks.WorkflowHistoryTaskId) as workflowHistoryTaskID
,trim(whtasks.WorkflowHistoryId) as workflowhistoryID
,trim(f.FileId) as fileID
,trim(doc.DocumentId) as documentID
,trim(fpol.DepartmentId) as fpolDepartmentID
--,trim(cu.DepartmentID) as cuDepartmentID
--,trim(uu.DepartmentID) as uuDepartmentID
,trim(fpol.CompanyID) as companyID
,trim(fpol.EffectiveDate) as effectiveDate
,trim(fpol.ExpirationDate) as expirationDate
,trim(whtasks.Initiated) as initiated
,trim(wsteps.WorkflowID) as workflowID
,trim(wsteps.WorkflowStepID) as workflowStepID
,trim(whtasks.DueDate) as dueDate
,trim(whtasks.DueDateDate) as dueDateDate
,trim(f.AgencyContactId) as agencyContactID
,trim(sr.RegionId) as srRegionID
,trim(r.RegionID) as fileRegionID
--,trim(whtasks.CompletedDate) as completedDate
,trim(whtasks.Completed) as completed
,trim(f.AgencyId) as agencyID
,trim(whtasks.AssignedToUserId) as assignedToUserID
,trim(whtasks.CompletedByUserId) as completedByUserID
,trim(wsteps.ExcludeFromMaster) as excludedFromMaster
,trim(whtasks.LastModified) as lastModified
--,trim(doc.Deleted) as docDeleted
--,trim(fpol.Deleted) as fpolDeleted
--,trim(f.Deleted) as fDeleted
--Dim Data
,trim(f.FileNumber) as fileNumber
,trim(wsteps.DepartmentType) as departmentType
,trim(c.CompanyName) as companyName
,trim(w.WorkflowName) as workflowName
,trim(wsteps.StepName) as stepName
,trim(r.RegionName) as regionName --look into how this works when the fregion is different from the sr region
,trim(f.FileName) as fileName
,trim(whtasks.TaskNote) as taskNote
,trim(whtasks.CompletedStatus) completedStatus
,trim(a.AgencyName) as agencyName
,trim(uu.FullName) as assignedToUserName
,trim(cu.FullName) as completedByUserName
,COALESCE(trim(whtasks.Locked), '0') as locked
,trim(whtasks.CompletedType) as completedType
,trim(ss.State) as agencyState 
--calculations
,DATEDIFF(day, whtasks.Initiated,whtasks.Completed) AS DaysTakenToComplete
,Case when whtasks.Completed is null AND DueDate > '2023-01-01 00:00:00.000'
				AND coalesce(doc.Deleted, fpol.Deleted, f.Deleted, cast(0 as boolean))=0
			then 'Pending'
			when whtasks.Completed is NOT null 
				AND whtasks.Completed > '2024-01-01 00:00:00.000' 
				AND whtasks.DueDate > '2023-01-01 00:00:00.000'
				AND coalesce(doc.Deleted, fpol.Deleted, f.Deleted, cast(0 as boolean))=0
				AND  CASE
    				WHEN whtasks.CompletedStatus = 'Task Updated' OR COALESCE(whtasks.CompletedType, '') != 'terminated' THEN 1
    				ELSE 0
  					END = 1
			then 'Completed'
	end as CompletedOrPending
,CASE 
		WHEN whtasks.Completed IS NULL AND whtasks.DueDate < current_timestamp() THEN 'OSS' 
		WHEN whtasks.Completed IS NULL AND whtasks.DueDate > current_timestamp() THEN 'Active' 
		ELSE NULL
		END AS StandardStatus

,CASE WHEN wsteps.WorkflowStepID = 60 THEN coalesce(uu.DepartmentID,cu.DepartmentID, fpol.DepartmentID)
	        ELSE CASE
			 when whtasks.completed IS NOT NULL and
                    case when wsteps.DepartmentType = 'assigneddepartment' THEN fpol.DepartmentID
                    when wsteps.DepartmentType != 'assigneddepartment' then coalesce(wsteps.ResponsibleDepartmentId,fpol.DepartmentID) 
                    End is not null
                then
                    case when wsteps.DepartmentType = 'assigneddepartment' THEN fpol.DepartmentID
                    when wsteps.DepartmentType != 'assigneddepartment' then coalesce(wsteps.ResponsibleDepartmentId,fpol.DepartmentID) 
                    End
                end 
end as ResponsibleDepartmentId

 

FROM bronze.tb_dm_workflowhistorytasks_spark as whtasks
INNER JOIN bronze.tb_dm_WorkflowHistory_spark as wh on wh.WorkflowHistoryId = whtasks.WorkflowHistoryId
INNER JOIN bronze.tb_dm_WorkflowSteps_spark as wsteps on wsteps.WorkflowStepID = whtasks.WorkflowStepId
LEFT JOIN bronze.tb_shared_users_spark as uu on uu.UserId = whtasks.AssignedToUserId
LEFT JOIN bronze.tb_shared_users_spark cu on cu.UserId = whtasks.CompletedByUserId
INNER JOIN bronze.tb_dm_Document_spark as doc on doc.DocumentId = wh.DocumentId 
LEFT JOIN bronze.tb_dm_File_spark as f on f.FileId = doc.FileId
LEFT JOIN bronze.tb_dm_FilePolicies_spark as fpol ON fpol.FilePolicyId = doc.FilePolicyId
INNER JOIN bronze.tb_dm_Workflows_spark as w ON w.WorkflowID = wsteps.WorkflowId
LEFT JOIN bronze.tb_am_Agencies_spark as a ON a.AgencyId = f.AgencyId
LEFT JOIN joinIsMailing jim on jim.agencyid0= a.agencyid
LEFT JOIN bronze.tb_shared_States_spark AS ss ON ss.State = 
                    case when coalesce(isMailing0, isMailing1)=0 then State0 else State1 end
LEFT JOIN bronze.tb_shared_Regions_spark r ON r.RegionId = f.RegionId
LEFT JOIN bronze.tb_shared_Regions_spark AS sr ON sr.RegionId = ss.RegionId 
LEFT JOIN bronze.tb_shared_Companies_spark as c ON c.CompanyId = fpol.CompanyId
